<a href="https://colab.research.google.com/github/BigTMiami/AdaptOrDie/blob/main/Roberta_Domain_Training_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

Create an account on huggingface.
Create an access token -

How to share datasets (and Models) to hugging face
https://huggingface.co/docs/datasets/en/upload_dataset

Add Access to Colab Notebooks.  Add the access token you created to HF_TOKEN in secrets, per website guide
https://drlee.io/how-to-use-secrets-in-google-colab-for-api-key-protection-a-guide-for-openai-huggingface-and-c1ec9e1277e0

In [ ]:
!pip install datasets
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 52.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 71.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━

# Prepare Dataset

I created a very small local Amazon Review dataset and uploaded it to HuggingFace. You should be able to load it from "BigTMiami/small_amazon_review_text_dataset"

In [ ]:
from datasets import load_dataset
dataset = load_dataset("BigTMiami/small_amazon_review_text_dataset")

Generating train split:   0%|          | 0/3504 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3504 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
tokenizer.model_max_length
block_size = tokenizer.model_max_length
print(f"block_size:{block_size}")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

block_size:512


In [ ]:
def tokenize_function(examples):
    # We use `return_special_tokens_mask=True` because DataCollatorForLanguageModeling (see below) is more
    # efficient when it receives the `special_tokens_mask`.
    # return tokenizer(examples["text"], return_special_tokens_mask=True)
    return tokenizer(examples["text"])

In [ ]:
tokenized_datasets = dataset.map(
    tokenize_function, batched=True, num_proc=4, remove_columns=["text"]
)
tokenized_datasets["train"]

Map (num_proc=4):   0%|          | 0/3504 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/3504 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 3504
})

In [ ]:
#Show three examples
for i in range(3):
    print(dataset["train"]["text"][i])
    print(tokenized_datasets["train"]["input_ids"][i])

Great product and price!
[0, 19065, 1152, 8, 425, 328, 2]
Waaay too small. Will use for futur children!
[0, 771, 6621, 857, 350, 650, 4, 2290, 304, 13, 20136, 710, 408, 328, 2]
Stays vibrant after many washes
[0, 5320, 4113, 11577, 71, 171, 21, 5065, 2]


In [ ]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

tokenizer.decode(lm_datasets["train"][0]["input_ids"])

Map (num_proc=4):   0%|          | 0/3504 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/3504 [00:00<?, ? examples/s]

'<s>Great product and price!</s><s>Waaay too small. Will use for futur children!</s><s>Stays vibrant after many washes</s><s>My son really likes the pink. Ones which I was nervous about</s><s>Waaay too small. Will use for future child.</s><s>Relieved my Plantar Fascitis for 3 Days. Then the unbearable pain returned in full force. These were recommended by my Podiatrist.</s><s>This is my 6th pair and they are the best thing ever for my plantar fasciitis and resultant neuromas. Unfortunately, the ones I ordered from SmartDestination must be seconds as they kill my feet. The hard plastic insert rubs on the outside edges of my feet. I am unable to exchange them as I waited one day too late to use them in my walking shoes.</s><s>We have used these inserts for years.  They provide great support.</s><s>Pinnacle seems to have more cushioning so my husband likes them better. He\'s tried all the other Powerstep inserts as well as other brands. This is his must have insert. He is 6\'3" and weighs

# Train

In [ ]:
from transformers import AutoConfig, AutoModelForMaskedLM

In [ ]:
config = AutoConfig.from_pretrained("roberta-base")

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [ ]:
model = AutoModelForMaskedLM.from_config(config)

In [ ]:
from transformers import Trainer, TrainingArguments

In [ ]:
#!pip install accelerate -U

In [ ]:
training_args = TrainingArguments(
    f"small_training_test_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    # push_to_hub=True
)

In [ ]:
!pip freeze

In [ ]:
print(f"block_size:{tokenizer.model_max_length}")

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm_probability=0.15
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
    data_collator=data_collator,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,9.439722
2,No log,9.074154
3,No log,8.977187


TrainOutput(global_step=75, training_loss=9.43480712890625, metrics={'train_runtime': 35.4689, 'train_samples_per_second': 16.662, 'train_steps_per_second': 2.115, 'total_flos': 155589888807936.0, 'train_loss': 9.43480712890625, 'epoch': 3.0})

In [ ]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 7951.15
